In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


In [ ]:

try:
    # Ruta del archivo y carga de la imagen original
    ruta_actual = os.getcwd()
    ruta_imagen = os.path.join(ruta_actual, '../datos/image.png')
    imagen_color = cv2.imread(ruta_imagen)
    if imagen_color is None:
        raise FileNotFoundError(f"Imagen no localizada en: {ruta_imagen}")

    # Transformación a escala de grises
    imagen_gris = cv2.cvtColor(imagen_color, cv2.COLOR_BGR2GRAY)

    # Vista previa en escala de grises
    plt.figure(figsize=(6, 6))
    plt.imshow(imagen_gris, cmap='gray')
    plt.title('Escala de grises aplicada')
    plt.axis('off')
    plt.show()

    # Binarización usando el umbral automático de Otsu
    _, binarizada_otsu = cv2.threshold(imagen_gris, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Conversión al espacio HSV para segmentar colores específicos
    imagen_hsv = cv2.cvtColor(imagen_color, cv2.COLOR_BGR2HSV)
    amarillo_claro_inf = np.array([20, 100, 100])
    amarillo_claro_sup = np.array([40, 255, 255])
    mascara_amarilla = cv2.inRange(imagen_hsv, amarillo_claro_inf, amarillo_claro_sup)

    # Unión de binarización y segmentación por color
    mascara_final = cv2.bitwise_or(binarizada_otsu, mascara_amarilla)

    # Visualización de la máscara compuesta
    plt.figure(figsize=(6, 6))
    plt.imshow(mascara_final, cmap='gray')
    plt.title(' Máscara combinada (Otsu + Amarillo)')
    plt.axis('off')
    plt.show()

except Exception as e:
    print(f"⚠️ Se produjo un error durante el procesamiento: {e}")


In [ ]:
try:
    # Buscar contornos en la imagen binarizada combinada
    contornos_detectados, _ = cv2.findContours(mascara_final, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Copiar imagen para mostrar anotaciones finales
    imagen_final = imagen_color.copy()

    for indice, contorno in enumerate(contornos_detectados, start=1):
        # Área y perímetro de la figura
        superficie = cv2.contourArea(contorno)
        longitud = cv2.arcLength(contorno, True)

        # Centroide mediante momentos
        momentos = cv2.moments(contorno)
        if momentos['m00'] != 0:
            centro_x = int(momentos['m10'] / momentos['m00'])
            centro_y = int(momentos['m01'] / momentos['m00'])
        else:
            centro_x, centro_y = 0, 0

        # Clasificación según número de vértices aproximados
        forma_aproximada = cv2.approxPolyDP(contorno, 0.04 * longitud, True)
        lados = len(forma_aproximada)
        if lados == 3:
            tipo = 'Triangulo'
        elif lados == 4:
            x, y, w, h = cv2.boundingRect(forma_aproximada)
            proporcion = w / float(h)
            tipo = 'Cuadrado' if 0.95 <= proporcion <= 1.05 else 'Rectangulo'
        elif lados > 4:
            tipo = 'Circulo'
        else:
            tipo = 'Figura'

        # Dibujar contorno sobre la imagen
        cv2.drawContours(imagen_final, [contorno], -1, (0, 255, 0), 2)

        # Preparar texto con métricas
        texto_info = f"{tipo}\nA:{superficie:.0f} P:{longitud:.0f}"
        for i, linea in enumerate(texto_info.split('\n')):
            cv2.putText(imagen_final, linea, (centro_x - 40, centro_y + i*20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50, 50, 50), 2, cv2.LINE_AA)

    # Mostrar imagen final con métricas embebidas
    imagen_rgb = cv2.cvtColor(imagen_final, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 8))
    plt.imshow(imagen_rgb)
    plt.title('Figuras con contorno y métricas respectivas')
    plt.axis('off')
    plt.show()

except Exception as fallo:
    print(f"⚠️ Error durante el análisis visual de figuras: {fallo}")
